In [3]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import six
import tensorflow as tf

from edward.models import (
    Dirichlet, Categorical, Empirical, ParamMixture)

plt.style.use('ggplot')

In [10]:
def build_toy_dataset(D, V, K):
    """
    Builds a toy dataset of D documents.
    
    Args:
        D - Number of documents
        V - Size of vocabulary
        K - Number of topics
        
    Returns:
        w - List of 1D NumPy arrays containing
            word tokens for each document
        theta - 2D NumPy array containing document
                topic distributions
        phi - 2D NumPy array containing topic
               token distributions
    """
    # Draw number of tokens for each document
    N = np.random.randint(low=2,
                          high=20,
                          size=D)
    
    # Draw topic distributions for each document
    theta = np.random.dirichlet(alpha=np.ones(K) * 0.1, 
                                size=D)

    # Create topics (non-overlapping)
    phi_values = np.array([K * 1 / V] * int(V / K) + [0.0] * int(V - V / K))
    phi = np.zeros([K, V])
    for k in range(K):
        phi[k, :] = np.roll(phi_values, int(k * V / K))
    
    # Draw tokens for each document
    w, z = [None] * D, [None] * D
    for d in range(D):
        # Draw token topic assignments
        z[d] = np.array([np.random.choice(range(K), size=N[d], p=theta[d, :])])[0]
        # Draw tokens
        w[d] = np.zeros(N[d])
        for n in range(N[d]):
            w[d][n] = np.random.choice(range(V), size=1, p=phi[z[d][n], :]) 

    return N, w, z, theta, phi

In [11]:
###############
# DATA
###############

# Get toy dataset
D = 10
V = 30
K = 5
N_train, w_train, z_train, theta_train, phi_train = build_toy_dataset(D, V, K)


In [4]:
###############
# MODEL
###############

alpha = tf.ones(K) * 0.01
beta = tf.ones(V) * 0.01

phi = Dirichlet(concentration=beta, 
                 sample_shape=K)

theta, w, z = [None] * D, [None] * D, [None] * D
for d in range(D):
    theta[d] = Dirichlet(concentration=alpha)
    
    w[d] = ParamMixture(mixing_weights=theta[d], 
                        component_params={'probs': phi},
                        component_dist=Categorical,
                        sample_shape=len(w_train[d]),
                        validate_args=True)
    z[d] = w[d].cat


In [5]:
####################
#INFERENCE
####################

# Data vars
data_dict={}
for d in range(D):
    data_dict[w[d]] = w_train[d]

# Latent vars
latent_vars_dict = {}

T = 500
qphi = Empirical(tf.Variable(tf.zeros([T, K, V])))
latent_vars_dict[phi] = qphi

qz, qtheta = [None] * D, [None] * D
for d in range(D):
    qtheta[d] = Empirical(tf.Variable(tf.ones([T, K]) / K))
    latent_vars_dict[theta[d]] = qtheta[d]
    
    N = len(w_train[d])
    
    qz[d] = Empirical(tf.Variable(tf.zeros([T, N], dtype=tf.int32)))
    latent_vars_dict[z[d]] = qz[d]
    

# Proposal vars
proposal_vars_dict = {}

phi_cond = ed.complete_conditional(phi)
proposal_vars_dict[phi] = phi_cond

theta_cond, z_cond = [None] * D, [None] * D
for d in range(D):
    print('Building proposals for doc {} of {}'.format(d + 1, D))
          
    theta_cond[d] = ed.complete_conditional(theta[d])
    proposal_vars_dict[theta[d]] = theta_cond[d]
    
    z_cond[d] = ed.complete_conditional(z[d])
    proposal_vars_dict[z[d]] = z_cond[d]
    

# Inference procedure w/Gibbs sampling
inference = ed.Gibbs(latent_vars=latent_vars_dict,
                     proposal_vars=proposal_vars_dict,
                     data=data_dict)

inference.initialize(n_iter=T, n_print=10, logdir='log')

tf.global_variables_initializer().run()

for n in range(inference.n_iter):
    info_dict = inference.update()
    inference.print_progress(info_dict)
    
inference.finalize()

Building proposals for doc 1 of 10
Building proposals for doc 2 of 10
Building proposals for doc 3 of 10
Building proposals for doc 4 of 10
Building proposals for doc 5 of 10
Building proposals for doc 6 of 10
Building proposals for doc 7 of 10
Building proposals for doc 8 of 10
Building proposals for doc 9 of 10
Building proposals for doc 10 of 10
500/500 [100%] ██████████████████████████████ Elapsed: 203s | Acceptance Rate: 1.000


In [6]:
# def build_toy_dataset(D, V, N, K):

#     alpha = np.ones(K) * 0.2
#     eta = np.ones(V) * 0.2

#     theta = np.random.dirichlet(alpha, size=D)
#     beta = np.random.dirichlet(eta, size=K)

#     Z = np.array([np.random.choice(range(K), size=N, p=theta_p) for theta_p in theta])

#     W = np.zeros([D, N])
#     for d in range(D):
#         for n in range(N):
#             z = Z[d, n]
#             W[d, n] = np.random.choice(range(V), size=1, p=beta[z]) 

#     return W


# def build_toy_dataset_fixed(D, N):
    
#     K = 2
#     V = 50

#     alpha = np.ones(K) * 0.9
#     theta = np.random.dirichlet(alpha, size=D)
    
# #     eta_values = np.array([0.2] * int(V / K) + [0.0] * int(V - V / K))
# #     beta = np.zeros([K, V])
# #     for k in range(K):
# #         eta = np.roll(eta_values, int(k * V / K))
# #         beta[k, :] = np.random.dirichlet(eta)

#     beta_values = np.array([K * 1 / V] * int(V / K) + [0.0] * int(V - V / K))
#     beta = np.zeros([K, V])
#     for k in range(K):
#         beta[k, :] = np.roll(beta_values, int(k * V / K))
        
#     beta = np.array([beta[0, :], beta[-1, :]])

#     Z = np.array([np.random.choice(range(K), size=N, p=theta_p) for theta_p in theta])

#     W = np.zeros([D, N])
#     for d in range(D):
#         for n in range(N):
#             z = Z[d, n]
#             W[d, n] = np.random.choice(range(V), size=1, p=beta[z, :]) 

#     return W, K, V, beta

# def build_toy_dataset_fixed2(D, N):
    
#     K = 2
#     V = 50

#     alpha = np.ones(K) * 1.0
#     theta = np.random.dirichlet(alpha, size=D)

#     beta_values = np.array([5 * 1 / V] * int(V / 5) + [0.0] * int(V - V / 5))
#     beta = np.zeros([K, V])

#     beta[0, :] = beta_values
#     beta[1, :] = np.roll(beta_values, int(3 * V / K))

#     Z = np.array([np.random.choice(range(K), size=N, p=theta_p) for theta_p in theta])

#     W = np.zeros([D, N])
#     for d in range(D):
#         for n in range(N):
#             z = Z[d, n]
#             W[d, n] = np.random.choice(range(V), size=1, p=beta[z, :]) 

#     return W, K, V, Z, beta, theta


# def build_toy_dataset_fixed3(D, N):
    
#     K = 3
#     V = 30

#     alpha = np.ones(K) * 1.0
#     theta = np.random.dirichlet(alpha, size=D)

#     beta = np.zeros([K, V])
    
#     beta[0, 0] = 0.85
#     beta[0, 1] = 0.15
#     beta[1, 10] = 0.15
#     beta[1, 11] = 0.85
#     beta[2, 20] = 0.95
#     beta[2, 21] = 0.05

#     Z = np.array([np.random.choice(range(K), size=N, p=theta_p) for theta_p in theta])

#     W = np.zeros([D, N])
#     for d in range(D):
#         for n in range(N):
#             z = Z[d, n]
#             W[d, n] = np.random.choice(range(V), size=1, p=beta[z, :]) 

#     return W, K, V, Z, beta, theta


# def build_toy_dataset_variable(N, V):

#     D = len(N)
#     K = 3
    
#     alpha = np.ones(K) * 0.01
#     theta = np.random.dirichlet(alpha, size=D)

#     beta = np.zeros([K, V])
    
#     beta[0, 0] = 0.85
#     beta[0, 1] = 0.15
#     beta[1, 10] = 0.15
#     beta[1, 11] = 0.85
#     beta[2, 20] = 0.95
#     beta[2, 21] = 0.05
    
#     w, z = [0] * D, [0] * D
#     for d in range(D):
#         w[d] = np.zeros(N[d])
#         z[d] = np.array([np.random.choice(range(K), size=N[d], p=theta[d, :])])[0]
        
#         for n in range(N[d]):
#             w[d][n] = np.random.choice(range(V), size=1, p=beta[z[d][n], :]) 

#     return K, w, z, beta, theta

